# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import pickle

import nltk
nltk.download(['punkt', 'wordnet', 'omw-1.4'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\2100\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\2100\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\2100\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///.\data\DisasterResponse.db')
df = pd.read_sql('SELECT * FROM messages', engine)
X = df.message.values
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1).values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [6]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000253C1CAF280>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
Y_pred = pipeline.predict(X_test)

In [8]:
range_columns = range(len(Y[0])-1)
range_rows_test = range(len(Y_test)-1)
range_rows_pred = range(len(Y_pred)-1)

In [9]:
report = dict.fromkeys(range_columns, [])
for col in range_columns:
    Y_test_col = [Y_test[i][col] for i in range_rows_test]
    Y_pred_col = [Y_pred[i][col] for i in range_rows_pred]
    try:
        target_names = np.unique(Y_test_col).astype('str')
        report[col] = classification_report(Y_test_col, Y_pred_col, target_names=target_names)
    except:
        target_names = np.unique(Y_pred_col).astype('str')
        report[col] = classification_report(Y_test_col, Y_pred_col, target_names=target_names)

In [10]:
for k in report.keys():
    print(report[k])

              precision    recall  f1-score   support

           0       0.47      0.44      0.45      1572
           1       0.82      0.84      0.83      4924
           2       0.30      0.12      0.18        48

    accuracy                           0.74      6544
   macro avg       0.53      0.47      0.49      6544
weighted avg       0.73      0.74      0.74      6544

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      5469
           1       0.55      0.59      0.57      1075

    accuracy                           0.85      6544
   macro avg       0.73      0.75      0.74      6544
weighted avg       0.86      0.85      0.85      6544

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6517
           1       0.00      0.00      0.00        27

    accuracy                           0.99      6544
   macro avg       0.50      0.50      0.50      6544
weighted avg       0

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000253C1CAF280>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=DecisionTreeClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000253C1CAF280>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=DecisionTreeClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [12]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__min_samples_split': [2, 3, 4],
        'clf__estimator': [DecisionTreeClassifier(), RandomForestClassifier(), KNeighborsClassifier()]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train, Y_train)

In [ ]:
pickle.dump(cv, open('disaster_recovery_gridsearch.pkl', 'wb'))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv = pickle.load(open('disaster_recovery_gridsearch.pkl', 'rb'))

In [ ]:
Y_pred_cv = cv.predict(X_test)

In [ ]:
report_cv = dict.fromkeys(range_columns, [])
for col in range_columns:
    Y_test_col = [Y_test[i][col] for i in range_rows_test]
    Y_pred_col = [Y_pred_cv[i][col] for i in range_rows_pred]
    try:
        target_names = np.unique(Y_test_col).astype('str')
        report_cv[col] = classification_report(Y_test_col, Y_pred_col, target_names=target_names)
    except:
        target_names = np.unique(Y_pred_col).astype('str')
        report_cv[col] = classification_report(Y_test_col, Y_pred_col, target_names=target_names)

In [ ]:
for k in report_cv.keys():
    print(report_cv[k])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(pipeline, open('disaster_recovery_pipeline.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
prova